In [110]:
import keras
import cv2
import os
import glob
import numpy as np
import pandas as pd

In [5]:
# Load saliency and motion videos and split them into chunks
split = 5 # split the video every five seconds
path = '../'

In [31]:
def splitVideo(videos, split, tp):
    newdir = path+'datasets/content/'+tp+'splitby-'+str(split)+'-seconds'
    if os.path.exists(newdir):
        os.system('rm -rf '+newdir)
    os.makedirs(newdir)
    for video in videos:
        command = 'ffmpeg -i ../datasets/content/'+tp+'/'+video+' -reset_timestamps 1 -map 0 -segment_time '+str(split)+' -f segment '+newdir+'/'+video[:-4]+'%03d.mp4'
        os.system(command)

In [36]:
def extractImages(videos, split, tp):
    newdir = path+'datasets/content/'+tp+'splitby-'+str(split)+'-seconds'
    if os.path.exists(newdir):
        os.system('rm -rf '+newdir)
    os.makedirs(newdir)
    for video in videos:
        os.system('ffmpeg -i ../datasets/content/'+tp+'/'+video+' -vf fps=30 '+newdir+'/'+video[:-4]+'%d.png')

In [37]:
saliVideos = os.listdir(path+'datasets/content/saliency/')
motionVideos = os.listdir(path+'datasets/content/motion/')

extractImages(saliVideos, split, 'saliency')
extractImages(motionVideos, split, 'motion')

In [ ]:
# make both saliency and motion map videos as numpy arrays
def makeVideoNumpy(videos, split, tp):
    fps = 30
    totalFrames = 1800
    imagesToRead = fps*split # total images to read based on split
    newdir = path+'datasets/content/'+tp+'Images'
    for video in videos:
        count = 1
        for i in range(1, totalFrames+1, imagesToRead):
            npArray = newdir+'/'+video[:-4]+str(count)+'.npy'
            count += 1
            tempArray = []
            for j in range(imagesToRead):
                imageName = newdir+'/'+video[:-4]+str(i+j)+'.png'
                image = cv2.imread(imageName, 0)
                tempArray.append(image)
            #print(np.array(tempArray).shape)
            np.save(npArray, np.array(tempArray))
makeVideoNumpy(saliVideos, split, 'saliency')
makeVideoNumpy(motionVideos, split, 'motion')

In [133]:
# make tiles information as numpy arrays
def makeTilesNumpy(videosTileData, split):
    files = glob.glob('../datasets/sensory/tile/*01*.csv')
    fps = 30
    totalFrames = 1800
    imagesToRead = fps*split # total images to read based on split
    for video, f in zip(videosTileData, files):
        count = 1
        for i in range(1, totalFrames+1, imagesToRead):
            npArray = f[:-8]+str(count)+'_split_by_'+str(split)+'_seconds.npy'
            tempArray = []
            for j in range(imagesToRead):
                tempArray.append(video[i+j-1])
            #print(np.array(tempArray).shape)
            np.save(npArray, np.array(tempArray))
            count += 1

In [ ]:
# encode tile information into images in banary fashion for convenience
def encodeTileInfoToImages():
    files = glob.glob('../datasets/sensory/tile/*01*.csv')
    tileData = []
    for f in files:
        headData = np.zeros((1800, 200))
        lines = open(f, 'r')
        lines = lines.readlines()[1:]
        tiles = []
        for line in lines:
            line = line.strip().split(',')[1:]
            line = [int(i) for i in line]
            tiles.append(line)
        for i, tile in enumerate(tiles):
            for j in tile:
                headData[i][j-1] = 255
        tileData.append(headData)
    return tileData
tileData = encodeTileInfoToImages()
makeTilesNumpy(tileData, split)